In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import csv
from timeit import default_timer as timer
from numba import jit, njit
from numba.typed import List
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv('bank_data_train.csv', skiprows=1)
trainArray = df.to_numpy()

file = open('bank_data_train.csv')
reader = csv.reader(file);
file.close()

df = pd.read_csv('bank_data_test.csv', skiprows=1)
finalTestingData = df.to_numpy()


In [3]:
def change_str_to_index(dataset, columnIndex):
    unique = list()
    for row in dataset:
        s = str(row[columnIndex])
        if (s == 'nan'):
            continue
        s = s.lower()
        if (s not in unique):
            unique.append(s)
        row[columnIndex] = unique.index(s)

def change_columns_to_int(dataset, columns):
    for column in columns:
        change_str_to_index(dataset, column)

#separate the data into classes according to the last value in each row
def separate_by_class(dataset):
	separated = dict()
	for i in range(len(dataset)):
		vector = dataset[i]
		class_value = vector[-1]
		if (class_value not in separated):
			separated[class_value] = list()
		separated[class_value].append(vector)
	return separated

In [4]:
#prep data
change_columns_to_int(trainArray, [13, 19, 24, 25, 27, 28, 30, 36, 39, 42, 53, 66, 88])

In [5]:
#split data to classes
classes = separate_by_class(trainArray)

In [6]:
import random
testSet = []
trainSet = []

for index in classes:
    size = int(len(classes[index]) * 0.2)
    print(len(classes[index]), size)
    random.shuffle(classes[index])
    testSet.extend(classes[index][:size])
    trainSet.extend(classes[index][size:])
    print(len(classes[index]), size)
random.shuffle(testSet)
random.shuffle(trainSet)
print(len(testSet))
print(len(trainSet))

326264 65252
326264 65252
28925 5785
28925 5785
71037
284152


In [22]:
train_x = [row[:-1] for row in trainSet]
train_y = [row[-1] for row in trainSet]
test_x = [row[:-1] for row in testSet]
test_y = [row[-1] for row in testSet]

In [28]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(train_x)

In [30]:
train_x_imp = imp.transform(train_x)

In [35]:
classifier = MLPClassifier(random_state=1, max_iter=300).fit(train_x_imp[:100000], train_y[:100000])

In [58]:
start = timer()
n = len(test_x)
v = 0
for i in range(n):
    test_imp = imp.transform(test_x[i].reshape(1, -1))
    prediction = classifier.predict(test_imp)
    if (prediction[0] == test_y[i]):
        v += 1
print("time: ", timer() - start)
print("accuracy = ", v / float(n))

time:  66.41475630000059
accuracy =  0.918563565465884


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer


X_train = [[0, 0, np.nan], [np.nan, 1, 1]]
Y_train = [0, 1]
X_test_1 = [0, 0, np.nan]
X_test_2 = [0, np.nan, np.nan]
X_test_3 = [np.nan, 1, 1]

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_train)

# Impute our data, then train
X_train_imp = imp.transform(X_train)
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train_imp, Y_train)

for X_test in [X_test_1, X_test_2, X_test_3]:
    # Impute each test item, then predict
    X_test_imp = imp.transform(X_test)
    print(X_test, '->', clf.predict(X_test_imp))

# Results
[0, 0, nan] -> [0]
[0, nan, nan] -> [0]
[nan, 1, 1] -> [1]

In [ ]:
X, y = make_classification(n_samples=100, random_state=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,random_state=1)

In [ ]:
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

In [ ]:
clf.predict_proba(X_test[:1])

In [ ]:
clf.predict(X_test[:5, :])

In [ ]:
clf.score(X_test, y_test)